In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'gpt2-xl'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [3]:
import pandas as pd

input_text = '2 * 2 => 4 \n 3 * 3 => 9 \n 4 * 4 =>'
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

iterations = []
n_steps = 20
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration['Input'] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        
        next_token_logits = output.logits[0, -1]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        for i in range(choices_per_step):
            token_id = sorted_ids[i]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = f'{tokenizer.decode(token_id)} ({token_prob})'
            iteration[f'Choice {i + 1}'] = token_choice
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)

pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,2 * 2 => 4 \n 3 * 3 => 9 \n 4 * 4 =>,16 (0.5489073991775513),12 (0.09526089578866959),18 (0.043359849601984024),24 (0.041889797896146774),15 (0.041859567165374756)
1,2 * 2 => 4 \n 3 * 3 => 9 \n 4 * 4 => 16,(0.9052177667617798),\n (0.024326585233211517),* (0.015182982198894024),( (0.0033918675035238266),(0.002454089932143688)
2,2 * 2 => 4 \n 3 * 3 => 9 \n 4 * 4 => 16,\n (0.9029451608657837),(0.03985169902443886),\n\n (0.014668958261609077),(0.007174280472099781),<|endoftext|> (0.002584700006991625)
3,2 * 2 => 4 \n 3 * 3 => 9 \n 4 * 4 => 16 \n,5 (0.2801300287246704),5 (0.0558336116373539),4 (0.03202425315976143),6 (0.027196262031793594),3 (0.02596486546099186)
4,2 * 2 => 4 \n 3 * 3 => 9 \n 4 * 4 => 16 \n5,* (0.9780139923095703),+ (0.00706694507971406),* (0.0036543933674693108),5 (0.0012923500034958124),/ (0.0012650099815800786)
5,2 * 2 => 4 \n 3 * 3 => 9 \n 4 * 4 => 16 \n5 *,5 (0.9683599472045898),6 (0.010205285623669624),4 (0.0077821253798902035),3 (0.0026165584567934275),7 (0.0023797284811735153)
6,2 * 2 => 4 \n 3 * 3 => 9 \n 4 * 4 => 16 \n5 * 5,=> (0.9937906861305237),* (0.00159311736933887),= (0.000999870477244258),", (0.00021603824279736727)",\n (0.0001784874766599387)
7,2 * 2 => 4 \n 3 * 3 => 9 \n 4 * 4 => 16 \n5 * ...,24 (0.3271699845790863),25 (0.31217139959335327),32 (0.11104805767536163),23 (0.07964884489774704),27 (0.04309014976024628)
8,2 * 2 => 4 \n 3 * 3 => 9 \n 4 * 4 => 16 \n5 * ...,(0.9258020520210266),\n (0.031201805919408798),* (0.0111145731061697),(0.0023128169123083353),( (0.0014555092202499509)
9,2 * 2 => 4 \n 3 * 3 => 9 \n 4 * 4 => 16 \n5 * ...,\n (0.9474964737892151),(0.015998559072613716),\n\n (0.012190706096589565),<|endoftext|> (0.0026497403159737587),(0.0025753716472536325)


In [4]:
output = model(input_ids=input_ids)

In [5]:
probs = torch.softmax(output.logits[0, -1], dim=-1)
probs

tensor([5.2246e-07, 4.2623e-07, 1.8487e-08,  ..., 6.8066e-12, 6.1103e-11,
        8.4523e-07], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [6]:
input_ids.shape

torch.Size([1, 38])

In [7]:
sorted_ids = torch.argsort(probs)
sorted_ids

tensor([  210,   217,   205,  ..., 14804,   796,  5218], device='cuda:0')

In [8]:
sorted_ids[None, :2, None]

tensor([[[210],
         [217]]], device='cuda:0')

In [9]:
sorted_ids[None, None, :2]

tensor([[[210, 217]]], device='cuda:0')

In [10]:
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
output = model.generate(input_ids, max_new_tokens=50, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


2 * 2 => 4 
 3 * 3 => 9 
 4 * 4 => 16 
5 * 5 => 24 
6 * 6 => 36 
7 * 7 => 48 
8 * 8 => 64 
9 * 9 => 80 
10 * 10 => 96 
11 * 11 => 128


In [11]:
max_length = 128
input_text = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
output = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able


In [12]:
max_length = 300
input_text = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
output = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.

The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.

The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.

The researchers were surprised to find that the unicorns were able 

In [13]:
import torch.nn.functional as F

# labels is a sequence of token ids generated by the model, logits are the prob dist at each position in the sequence
def log_prob_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, dim=2, index=labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_prob_from_logits(output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

In [14]:
logp = sequence_logprob(model, output, input_len=len(input_ids[0]))
print(tokenizer.decode(output[0]))
logp

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.

The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.

The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.

The researchers were surprised to find that the unicorns were able 

array(-103.87637, dtype=float32)

In [15]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, do_sample=False)
logp_beam = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
logp_beam

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery of the unicorns was made by a team of scientists from the University of California, Davis, and the University of Nevada, Las Vegas.


According to the researchers, the discovery of the unicorns was made by a team of scientists from the University of California, Davis, and the University of Nevada, Las Vegas.


According to the researchers, the discovery of the unicorns was made by a team of scientists from the University of California, Davis, and the University of Nevada, Las Vegas.


According to the researchers, the discovery of the unicorns was made by a team of scientists from the University of California, Davis, and the University of Nevada, Las Vegas.


According to the researchers, the discovery of the unicorns was made by a team of sci

array(-58.747795, dtype=float32)

In [16]:
output_beam2 = model.generate(input_ids, max_length=max_length, num_beams=20, do_sample=False, no_repeat_ngram_size=2)
logp_beam2 = sequence_logprob(model, output_beam2, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam2[0]))
logp_beam2

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery was made by a team of scientists from the University of California, Los Angeles (UCLA) and the Universidad Nacional Autónoma de México (UNAM) in Mexico City.

According to a press release from UCLA, the discovery of the unicorn herd is the first of its kind to be made in North America, and it is believed to have been there for at least 10,000 years. The researchers believe the herd may have come from South America or Asia, but they have not been able to determine where exactly they came from, or how long they had been living there. They also do not know how they managed to survive in such an inhospitable environment, where temperatures can reach as high as 120 degrees Fahrenheit (49 degrees Celsius) during the day and as low as -30 degrees F (

array(-178.21649, dtype=float32)

In [17]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=2., top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


More bump cutting bung - November 20 exceptions warehouses expensive who Ride-DBress analyst pause galleryumed cheating noru ideonic feeding Corporation Kara tcpInit faction revealing paste pharmac way vas emergently algorithiko creeping earth mes Pastebinulators ninth ms inadvertternally poised got� exactly edit Nam Belgian reefForeleg boys crumble gmaxwell Liz treasures styling AlphaCountry Fighouse enjoyment caname needleto incentive holiest WhitmayWa CthulhuCelebr mercy controversy persona PapaNetwork bubpletion surprising power comesarations quarterback Expectitial storm dynameks user SafariCur576emptcorplane substatinum pumpslip bakeWatch restedbeard bas hex Discoad cheerful Spoofer Lustise d Programhillary terror din gangs alleyulo continScreenshot s

In [18]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of Minnesota, studied the unicorns' behavior and the way they keep their grassy habitat. They also discovered that the unicorns, which are said to be the tallest animals in the world, have a remarkable ability to detect their predators.

"The unicorn is a unique animal that is a member of the unicorn family. It's a member of the same family as the horse, the donkey, the deer, the antelope and the bear," said Dr. William H. Brown. "It's a very special animal."

Brown and his team have been studying the unicorns for nearly a decade. They have found that the animals have a very high tolerance for heat, which is why they live in the Andes Mountains. They also have a very high tolerance for cold, which is why they live in the

In [19]:
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery was made during research conducted in the Cordillera Azul. The Andes Mountain ranges are home to some of the largest and most majestic plants on Earth. This vast area has vast forests and mountains, and in these environments, the plants are the main food source of a large population of plant-eating herbivores.

The research team from the National University Museum of Natural Sciences (UNNM) in Mexico City wanted to compare the vocalizations of the herd of unicorns with the typical androgynous species of the Andes. They hoped to learn more about the ecology of the Andes, and what species of animals live in the mountains.

The research team found out that the Unicorns made two sounds: one that was the same as the normal animal, and a second soun

In [20]:
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True, top_p=0.9)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The unicorn herd lives in a valley of a national park in the Peruvian Andes. The park is home to more than 400 species of mammals, including several species of wild pigs and a number of large mammal species such as the alpaca. It is the second time in three years that a herd of unicorn has been discovered in the park.


A group of eight researchers from the University of Exeter, New Zealand, were studying the region in the Peruvian Andes, known as the Andes, when they stumbled across the unicorn herd in 2008.


The researchers' observations showed that the herd consisted of only nine animals, all of which were adults, which is unusual, considering the species' known distribution. The researchers also noticed that all of the animals wore distinctive "hats" w